# Исследование надежности заемщиков.

Задача:
Необходимо найти зависимости и определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Обзор данных

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/datasets/data.csv')
df.info() # Общий обзор таблицы
display(df.head(10)) # Вывод первых десяти строк таблицы
display(df.isna().sum()) # Обзор на наличие пропусков в данных
display(df.duplicated().sum()) # Обзор на наличие дубликатов в данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

54

12 столбцов с 21525 строк данных, два из которых выделяются по меньшему количеству значений.
По предварительной оценке видно:
- в двух столбцах 'days_employed' и 'total_income' присутствуют пропуски в данных в количестве 2174.
- есть наличие 54 строк-дубликатов.
- столбец 'total_income' хранит данные о сумме доходов клиентов, а значит формат данных должен быть числовым, при том целочисленным (с округлением до рубля).
- неверный регистр записи данных.
- присутствуют отрицательные значения, несоответствующие смысловому значению данных столбца, так же следует провести более детальный осмотр на наличие других аномалий в данных.

### Шаг 2.1 Заполнение пропусков

In [4]:
days_median = df['days_employed'].median()
total_median = df['total_income'].median()
df['days_employed'] = df['days_employed'].fillna(days_median)
df['total_income'] = df['total_income'].fillna(total_median)
df.isna().sum() # Контрольный вывод на проверку отсутствия пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

1. Используем функцию isna() для поиска пропущенных значений и применяем его ко всему датафрэйму, а функцией sum() находим их общее количество по каждому из 12-ти столбцов (df.isna().sum()). Результатом является следующие данные: в двух столбцах 'days_employed' и 'total_income' было найдено 2174 пропуска, это 11.23% от общего количества (общее число значений в каждом из двух столбцов составляет 19351).
2. Скорее пропуски несут естественный характер. Все пропущенные значения в двух столбцах относятся к одним и тем же людям, то есть строки и их количество в обоих столбцах совпадают. Люди к которым относятся пропущенные значения на данный момент являются безработными т.к. столбцы отвечают за стаж и за доход.
3. Медиана более реальна, и т.к. здесь ассиметричное распределение данных, она отражает среднюю тенденцию.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
df['days_employed'] =  df['days_employed'].abs()
df['children'] =  df['children'].abs() #Убираем все отрицательные значения в столбцах

In [6]:
df['purpose'] = df['purpose'].replace('ремонт жилью', 'ремонт жилья')
df['education_id'] = df['education_id'].astype('str')
df['education_id'] = df['education_id'].replace(to_replace=[0, 1, 2, 3], 
    value=['A', 'C', 'B', 'D'])

In [7]:
df.head(10) # Контрольный вывод на проверку того, что отрицательные значения столбца исправлены

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


1. 'ремонт жилью' является одной из аномалий в значениях, где причиной послужил человеческий фактор - простая опечатка. Тоже самое можно сказать про значение '-1' в столбце 'children'.
2. Что касается столбца 'education_id' я сменил ему тип данных и поменял соотношение классификации. Ранее порядок был таков: высшее-0, среднее-1, неполное высшее-2, начальное-3 я посчитал что среднее и неоконченное высшее надо поменять местами, т.к. по смыслу это будет верно. Ошибка относится к человеческому фактору, где изначально определили неверный порядок присвоения категории.
3. Строка с отрицательным значением в столбце 'children' так же является разовой аномалией и опечаткой.
4. Данные в столбце 'days_employed' имеют отрицательный знак только если значение меньше 10000(около того). Возможно это ошибка системы, где изначально выбрана неправильная система подсчета данных, неверное распределение дат к примеру.

### Шаг 2.3. Изменение типов данных.

In [8]:
df['total_income'] = df['total_income'].astype('int')
df.info() # Контрольный вывод на проверку того, что тип столбца действительно изменился

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  object 
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [9]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() # строка для проверки, что все строки дубликаты действительно удалены

0

1. Предыдущие выводы первых(можно и последних) строк всей таблицы указали на проблемные столбцы. Метод str.lower() помогает привести все значения к нижнему регистру (строчные буквы), что здесь и необходимо.
2. Неверный регистр записи значений связан с ошибкой исполнителя при вводе данных, возможно клиент не обратил внимание на это. В таком случаи можно научить систему приводить все буквы к строчному типу.
3. Появление строк дубликатов можно отнести к человеческому фактору(ошибка внесения данных). Так же существует крайне низкая вероятность полного совпадения всех параметров клиента.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [10]:
df_new_education = pd.DataFrame(data=df, columns=['education_id', 'education'])
df_new_family_status = pd.DataFrame(data=df, columns=['family_status_id', 'family_status'])
df = df.drop(['education', 'family_status'], 1) # Удаление столбцов из исходного датафрейма
display(df_new_education.head(10)) # Проверка создания нового датафрейма
display(df_new_family_status.head(10)) # Проверка создания нового датафрейма

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


### Шаг 2.6. Категоризация дохода.

In [11]:
def total_income_category(total_income):
    try:
        if total_income <= 30000:
            return 'E'
        if total_income <= 50000:
            return 'D'
        if total_income <= 200000:
            return 'C'
        if total_income <= 1000000:
            return 'B'
        if total_income > 1000000:
            return 'A'
    except:
        return 'Кажется значение вне диапозона, проверьте логическую конструкцию'
    
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head(10) # Проверка создания столбца с категориями

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [19]:
df['purpose'].unique() # Необходимая строка для оценки всех уникальных значений в столбце
def replace_purpose_action(action):
    duplicates_car = ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 
                  'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 
                  'сделка с автомобилем', 'на покупку автомобиля']
    duplicates_estate = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости',
                    'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 
                     'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 
                    'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 
                     'операции со своей недвижимостью', 'покупка своего жилья', 'покупка жилья для сдачи', 'ремонт жилья']
    duplicates_wedding = ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']
    duplicates_study = ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования',
                       'получение дополнительного образования', 'получение высшего образования', 'профильное образование',
                       'высшее образование', 'заняться высшим образованием']
    try:
        if action in duplicates_car:
            return 'операции с автомобилем'
        if action in duplicates_estate:
            return 'операции с недвижимостью'
        if action in duplicates_wedding:
            return 'проведение свадьбы'
        if action in duplicates_study:
            return 'получение образования'
    except:
        return 'Что-то пошло не так, нужно проверить логическое условие'
    
df['purpose_category'] = df['purpose'].apply(replace_purpose_action)
df.head(50) # Проверка создания столбца с категориями

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

In [13]:
children_category_grouped = df.groupby('children').agg({'debt' : ['count', 'sum']})
children_category_grouped['children_result'] = children_category_grouped['debt']['sum'] / children_category_grouped['debt']['count']
display(children_category_grouped)
children_piv = children_category_grouped.pivot_table(index=['children'], values='children_result')
display(children_piv) #Сводная таблица результатов

debt       children_result
          count   sum                
children                             
0         14091  1063        0.075438
1          4855   445        0.091658
2          2052   194        0.094542
3           330    27        0.081818
4            41     4        0.097561
5             9     0        0.000000
20           76     8        0.105263

children
0     0.075438
1     0.091658
2     0.094542
3     0.081818
4     0.097561
5     0.000000
20    0.105263
Name: children_result, dtype: float64

0.9.0.1 Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

0.9.0.2 Вывод 1:

Лучшие показатели у кредиторов которые не имеют детей. Показатели по позиции в 5 детей можно считать неполными т.к. таких клиентов всего лишь 9 человек. В целом тенденция ухудшения с увеличением количества детей однородна, за исключением позиции '3'. 
На основании этого, теорию о влиянии количества детей можно считать верной лишь частично.

In [14]:
#df_family_status_prepared = df.merge(df_new_family_status, on='family_status_id', how='left')
#df_family_status_piv = df_family_status_prepared.pivot_table(index=['family_status'], values='debt', aggfunc='sum')

family_status_grouped = df.groupby('family_status_id').agg({'debt' : ['count', 'sum']})
family_status_grouped['family_status_result'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count']
display(family_status_grouped)
family_status_piv = family_status_grouped.pivot_table(index=['family_status_id'], values='family_status_result')
display(family_status_piv) #Сводная таблица результатов

debt      family_status_result
                  count  sum                     
family_status_id                                 
0                 12339  931             0.075452
1                  4151  388             0.093471
2                   959   63             0.065693
3                  1195   85             0.071130
4                  2810  274             0.097509

family_status_id
0    0.075452
1    0.093471
2    0.065693
3    0.071130
4    0.097509
Name: family_status_result, dtype: float64

0.9.0.3  Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

0.9.0.4  Вывод 2:

In [15]:
total_income_category_grouped = df.groupby('total_income_category').agg({'debt' : ['count', 'sum']})
total_income_category_grouped['total_income_result'] = total_income_category_grouped['debt']['sum'] / total_income_category_grouped['debt']['count']
display(total_income_category_grouped)
total_income_piv = total_income_category_grouped.pivot_table(index=['total_income_category'], values='total_income_result')
display(total_income_piv) #Сводная таблица результатов

debt       total_income_result
                       count   sum                    
total_income_category                                 
A                         25     2            0.080000
B                       5041   356            0.070621
C                      16016  1360            0.084915
D                        350    21            0.060000
E                         22     2            0.090909

total_income_category
A    0.080000
B    0.070621
C    0.084915
D    0.060000
E    0.090909
Name: total_income_result, dtype: float64

0.9.0.5 Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

0.9.0.6  Вывод 3:

Нет, теория не подтвердилась, никакой зависимости не наблюдается.Единственное можно сказать о пиковом значении, которое пришлось на категорию 'E' здесь доход кредиторов самый низкий, и как следствие самое большой процент нарушителей.

In [16]:
purpose_category_grouped = df.groupby('purpose_category').agg({'debt' : ['count', 'sum']})
purpose_category_grouped['purpose_category_result'] = purpose_category_grouped['debt']['sum'] / purpose_category_grouped['debt']['count']
display(purpose_category_grouped)
purpose_category_piv = purpose_category_grouped.pivot_table(index=['purpose_category'], values='purpose_category_result')
display(purpose_category_piv) #Сводная таблица результатов

debt      purpose_category_result
                         count  sum                        
purpose_category                                           
операции с автомобилем    4306  403                0.093590
операции с недвижимостью  9530  686                0.071983
получение образования     4013  370                0.092200
проведение свадьбы        2324  186                0.080034

purpose_category
операции с автомобилем      0.093590
операции с недвижимостью    0.071983
получение образования       0.092200
проведение свадьбы          0.080034
Name: purpose_category_result, dtype: float64

0.9.0.7 Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

0.9.0.8  Вывод 4:

Кредиторы, чей целью был заем на операции с автомобилем и образование, примерно в 9% случаях нарушают условие договора с банком, а в случаи свадебных мероприятий 8%. Самыми ответственными оказались заемщики на операции с недвижимостью, чей процент нарушителей составил около 7%.
Исследование целей кредита носит равномерный и некритичный характер. Общая разница составляет лишь 2%.

## Общий вывод:

Проанализировав все данные получили следующие выводы:
1. Теория влияния семейного положения на возврат кредита в срок подтвердилась.
2. Теория влияния уровня дохода на возврат кредита в срок не подтвердилась.
3. Теории о количестве детей и целях кредита подтвердились частично.
4. Так же стоит обращать внимание на критичный показатель дохода в категории 'E', который даже с точки зрения простой логики несет существенные риски.
5. Интересный момент, в случаи с займом на мероприятия связанных со свадьбой, все 100% нарушителей входят в категорию людей живущих гражданским браком. Следует быть осторожным выдавая кредит людям с таким семейным статусом на подобные цели.

In [17]:
result_family_status_id = df.pivot_table(index=['family_status_id'], columns='purpose_category', values='debt', aggfunc='sum')
display(result_family_status_id)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
family_status_id,,,,
0,229.0,427.0,216.0,NaN
1,51.0,78.0,60.0,186.0
2,20.0,25.0,15.0,NaN
3,21.0,41.0,17.0,NaN
4,82.0,115.0,62.0,NaN


In [18]:
result_children = df.pivot_table(index=['children'], columns='purpose_category', values='debt', aggfunc='sum')
display(result_children)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
children,,,,
0,243,420,229,115
1,104,168,90,51
2,48,83,46,12
3,5,11,4,5
4,1,2,0,0
5,0,0,0,0
20,2,2,1,3
